In [86]:
import pandas as pd
import numpy as np
from packages import wrappers, models, ta
from sklearn.linear_model import LinearRegression

In [87]:
asset_df = wrappers.get_df_from_cryptowatch(pair='ethusd')

In [88]:
day_predicting = '2021-08-16'
num_days = 365

In [89]:
prepped_df = models.add_TA(asset_df)

In [90]:
prepped_df = models.prepare_for_model(asset_df, day_predicting, num_days)

In [91]:
prepped_df

,Target,Close,Volume,ATR_14,OBV
Date,,,,,
2020-08-15,433.67,432.39,2.006905e+07,25.852238,2.023126e+06
2020-08-16,431.35,433.67,2.717136e+07,25.792078,2.087150e+06
2020-08-17,422.62,431.35,3.055223e+07,25.873358,2.016556e+06
2020-08-18,407.57,422.62,3.295368e+07,25.603118,1.938928e+06
2020-08-19,416.20,407.57,3.725911e+07,26.143610,1.847700e+06
...,...,...,...,...,...
2021-08-11,3048.80,3164.50,1.602499e+08,195.612514,5.222545e+06
2021-08-12,3324.02,3048.80,1.848556e+08,199.969477,5.162558e+06
2021-08-13,3266.38,3324.02,1.681553e+08,206.524514,5.214775e+06


In [92]:
Xtrain, Xtest, ytrain, ytest = models.get_train_test(prepped_df)
Xtrain

,Close,Volume,ATR_14,OBV
Date,,,,
2020-08-15,432.39,2.006905e+07,25.852238,2.023126e+06
2020-08-16,433.67,2.717136e+07,25.792078,2.087150e+06
2020-08-17,431.35,3.055223e+07,25.873358,2.016556e+06
2020-08-18,422.62,3.295368e+07,25.603118,1.938928e+06
2020-08-19,407.57,3.725911e+07,26.143610,1.847700e+06
...,...,...,...,...
2021-08-10,3141.69,2.141756e+08,198.881938,5.172652e+06
2021-08-11,3164.50,1.602499e+08,195.612514,5.222545e+06
2021-08-12,3048.80,1.848556e+08,199.969477,5.162558e+06


In [93]:
lr = LinearRegression().fit(Xtrain, ytrain)
prediction = lr.predict(Xtest)[0][0]
actual = ytest.iloc[0, 0]

In [94]:
print('Prediction:', round(prediction, 2), 'or', round((prediction - prepped_df['Close'].iloc[[-1]][0]) / prediction * 100, 2), '%')
print('Actual:', actual, 'or', round((actual - prepped_df['Close'].iloc[[-1]][0]) / actual * 100, 2), '%')

Prediction: 3290.33 or -0.64 %
Actual: 3147.34 or -5.21 %


# Testing Documentation

In [95]:
# ?models.get_train_test